## **Installing librarys**

In [1]:
!pip install torch_geometric

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 8.1 MB/s eta 0:00:00


## **Imports**

In [2]:
import numpy as np
import torch
from torch_geometric.datasets import Planetoid
from torch_geometric.datasets import CitationFull
from torch_geometric.datasets import CoraFull
from torch_geometric.data import DataLoader
from torch.utils.data import Subset
from torch_geometric.transforms import ToSparseTensor
from torch_geometric.utils import train_test_split_edges
import torch.nn as nn
import torch.optim as optim
from torch_geometric.nn import MLP
from torch.utils.data import DataLoader
from torch.utils.data import TensorDataset
from torch.nn.functional import softmax
from torch_geometric.nn import GCNConv
import torch.nn.functional as F
from itertools import permutations
from itertools import product
from torch_geometric.nn import GATConv
from torch_geometric.nn import GATv2Conv

## **Download Datasets**

In [3]:
# Split the dataset into training, validation, and test sets
train_ratio, val_ratio, test_ratio = 0.7, 0.1, 0.2

In [4]:
# Download the dataset --> citeseer
dataset_cite = Planetoid(root='./cite', name='CiteSeer')

cite_classes = dataset_cite.num_classes
cite_features = dataset_cite.num_features

# Calculate the number of all nodes
nodes = dataset_cite[0].num_nodes

# Shuffle the indices to create a random split
indices = torch.randperm(nodes)

# Calculate size of each split
num_train = int(train_ratio * nodes)
num_val = int(val_ratio * nodes)
num_test = nodes - num_train - num_val

# Create a mask for each split
train_mask_cite = torch.zeros(nodes, dtype=torch.bool)
val_mask_cite = torch.zeros(nodes, dtype=torch.bool)
test_mask_cite = torch.zeros(nodes, dtype=torch.bool)

# Convert the mask indices into true for split selection
train_mask_cite[indices[: num_train]]=True
val_mask_cite[indices[num_train : num_train+num_val]]=True
test_mask_cite[indices[num_train+num_val :]]=True

# Apply the calculated masks into the dataset
dataset_cite[0].train_mask = train_mask_cite
dataset_cite[0].val_mask = val_mask_cite
dataset_cite[0].test_mask = test_mask_cite

print(dataset_cite[0])

Processing...


Data(x=[3327, 3703], edge_index=[2, 9104], y=[3327], train_mask=[3327], val_mask=[3327], test_mask=[3327])


Done!


In [5]:
# Download the dataset --> corafull
dataset_corafull = CitationFull(root='./cora', name='cora')

cora_classes = dataset_corafull.num_classes
cora_features = dataset_corafull.num_features

# Calculate the number of all nodes
nodes = dataset_corafull[0].num_nodes

# Shuffle the indices to create a random split
indices = torch.randperm(nodes)

# Calculate size of each split
num_train = int(train_ratio * nodes)
num_val = int(val_ratio * nodes)
num_test = nodes - num_train - num_val

# Create a mask for each split
train_mask_cora = torch.zeros(nodes, dtype=torch.bool)
val_mask_cora = torch.zeros(nodes, dtype=torch.bool)
test_mask_cora = torch.zeros(nodes, dtype=torch.bool)

# Convert the mask indices into true for split selection
train_mask_cora[indices[: num_train]]=True
val_mask_cora[indices[num_train : num_train+num_val]]=True
test_mask_cora[indices[num_train+num_val :]]=True

dataset_cora = dataset_corafull[0]

# Apply the calculated masks into the dataset
dataset_cora.train_mask = train_mask_cora
dataset_cora.val_mask = val_mask_cora
dataset_cora.test_mask = test_mask_cora


dataset_cora.num_classes = dataset_corafull.num_classes
dataset_cora.cora_features = dataset_corafull.num_features

print(dataset_cora)

Processing...
Done!


Data(x=[19793, 8710], edge_index=[2, 126842], y=[19793], train_mask=[19793], val_mask=[19793], test_mask=[19793], num_classes=70, cora_features=8710)


## **DropEdge Method**

In [15]:
class DropEdgeLayer(nn.Module):
    def __init__(self, drop_rate):
        super(DropEdgeLayer, self).__init__()
        self.drop_rate = drop_rate

    def forward(self, edge_index):
        edge_mask = torch.bernoulli(torch.full((edge_index.size(1),), 1 - self.drop_rate)).bool()
        return edge_index[:, edge_mask]

class GCNTwoLayer_DropEdge(torch.nn.Module):
    def __init__(self, num_features, hidden_channels, num_classes, drop_rate, num_layers=2):
        super(GCNTwoLayer_DropEdge, self).__init__()
        self.num_layers = num_layers

        # Define GCN layers
        self.conv_layers = nn.ModuleList([GCNConv(num_features, hidden_channels)])
        self.drop_edge_layers = nn.ModuleList([DropEdgeLayer(drop_rate) for _ in range(num_layers - 1)])
        self.conv_layers.extend([GCNConv(hidden_channels, hidden_channels) for _ in range(num_layers - 1)])
        self.final_layer = GCNConv(hidden_channels, num_classes)

    def forward(self, data):
        x, edge_index = data.x, data.edge_index

        # Apply DropEdge for each layer
        for i in range(self.num_layers):
            if i > 0:
                edge_index = self.drop_edge_layers[i-1](edge_index)

            x = self.conv_layers[i](x, edge_index)
            x = F.relu(x)
            x = F.dropout(x, training=self.training)

        x = self.final_layer(x, edge_index)

        return F.log_softmax(x, dim=1)

In [22]:
dataset = dataset_cite
print('Dataset: CiteSeer')

# Define training parameters
# Define training parameters
layer_size = 16
epochs = 55
learning_rate = 0.008
drop_rate = 0.35

# Define model
model = GCNTwoLayer_DropEdge(dataset.num_features, layer_size, dataset.num_classes, drop_rate, num_layers=2)
optimizer = optim.Adam(model.parameters(), lr=learning_rate)
criterion = nn.CrossEntropyLoss()

print(f"Model with hidden size({layer_size}):\n")
print(f"...Training Start...")

# Training
model.train()

for epoch in range(epochs):
    optimizer.zero_grad()
    out = model(dataset)
    loss = criterion(out[dataset.train_mask], dataset.y[dataset.train_mask])
    loss.backward()
    optimizer.step()

    # Calculate training accuracy
    _, predicted = out.max(dim=1)
    correct = predicted[dataset.train_mask] == dataset.y[dataset.train_mask]
    train_acc = correct.sum().item() / dataset.train_mask.sum().item()

    # Printing Every 20 epochs
    if ((epoch+1) % 10) == 0 or epoch+1 == epochs:
        print(f"Epoch {epoch+1}: loss={loss.item()}, training accuracy={train_acc}")

# Evaluation
model.eval()
with torch.no_grad():
    logits = model(dataset)
    _, predicted = logits.max(dim=1)
    correct = predicted[dataset.val_mask] == dataset.y[dataset.val_mask]
    acc = correct.sum().item() / dataset.val_mask.sum().item()

print(f"...Training End...")
print(f"\nValidation accuracy= {acc}")

# Test Evaluation for the Best Model
model.eval()
with torch.no_grad():
    logits = model(dataset)
    _, predicted = logits.max(dim=1)
    correct = predicted[dataset.test_mask] == dataset.y[dataset.test_mask]
    acc = correct.sum().item() / dataset.test_mask.sum().item()

print(f"Test accuracy= {acc}")

Dataset: CiteSeer
Model with hidden size(16):

...Training Start...
Epoch 10: loss=1.105533480644226, training accuracy=0.575
Epoch 20: loss=0.6705427765846252, training accuracy=0.7916666666666666
Epoch 30: loss=0.3271563649177551, training accuracy=0.8666666666666667
Epoch 40: loss=0.3873106837272644, training accuracy=0.875
Epoch 50: loss=0.24747994542121887, training accuracy=0.925
Epoch 55: loss=0.32405534386634827, training accuracy=0.9083333333333333
...Training End...

Validation accuracy= 0.648
Test accuracy= 0.657


In [23]:
dataset = dataset_cora
print('Dataset: CoraFull')

# Define training parameters
# Define training parameters
layer_size = 16
epochs = 230
learning_rate = 0.008
drop_rate = 0.35

# Define model
model = GCNTwoLayer_DropEdge(dataset.num_features, layer_size, dataset.num_classes, drop_rate, num_layers=2)
optimizer = optim.Adam(model.parameters(), lr=learning_rate)
criterion = nn.CrossEntropyLoss()

print(f"Model with hidden size({layer_size}):\n")
print(f"...Training Start...")

# Training
model.train()

for epoch in range(epochs):
    optimizer.zero_grad()
    out = model(dataset)
    loss = criterion(out[dataset.train_mask], dataset.y[dataset.train_mask])
    loss.backward()
    optimizer.step()

    # Calculate training accuracy
    _, predicted = out.max(dim=1)
    correct = predicted[dataset.train_mask] == dataset.y[dataset.train_mask]
    train_acc = correct.sum().item() / dataset.train_mask.sum().item()

    # Printing Every 20 epochs
    if ((epoch+1) % 20) == 0 or epoch+1 == epochs:
        print(f"Epoch {epoch+1}: loss={loss.item()}, training accuracy={train_acc}")

# Evaluation
model.eval()
with torch.no_grad():
    logits = model(dataset)
    _, predicted = logits.max(dim=1)
    correct = predicted[dataset.val_mask] == dataset.y[dataset.val_mask]
    acc = correct.sum().item() / dataset.val_mask.sum().item()

print(f"...Training End...")
print(f"\nValidation accuracy= {acc}")

# Test Evaluation for the Best Model
model.eval()
with torch.no_grad():
    logits = model(dataset)
    _, predicted = logits.max(dim=1)
    correct = predicted[dataset.test_mask] == dataset.y[dataset.test_mask]
    acc = correct.sum().item() / dataset.test_mask.sum().item()

print(f"Test accuracy= {acc}")

Dataset: CoraFull
Model with hidden size(16):

...Training Start...
Epoch 20: loss=3.1894445419311523, training accuracy=0.20988812702995308
Epoch 40: loss=2.3731744289398193, training accuracy=0.4042583904727535
Epoch 60: loss=2.0079641342163086, training accuracy=0.48357993504150126
Epoch 80: loss=1.8192232847213745, training accuracy=0.52948394081559
Epoch 100: loss=1.701387643814087, training accuracy=0.5523637675929267
Epoch 120: loss=1.6571295261383057, training accuracy=0.5592204980151569
Epoch 140: loss=1.6148221492767334, training accuracy=0.5660772284373872
Epoch 160: loss=1.5593241453170776, training accuracy=0.5818837964633706
Epoch 180: loss=1.5439478158950806, training accuracy=0.5827499097798629
Epoch 200: loss=1.5004788637161255, training accuracy=0.594947672320462
Epoch 220: loss=1.4969053268432617, training accuracy=0.5891014074341393
Epoch 230: loss=1.472802758216858, training accuracy=0.6023818116203536
...Training End...

Validation accuracy= 0.6538655886811521
Tes

## **GCN with 8 layers**

In [24]:
class GCNWithDropEdge(nn.Module):
    def __init__(self, num_features, hidden_channels, num_classes, drop_rate, num_layers=8):
        super(GCNWithDropEdge, self).__init__()
        self.num_layers = num_layers

        self.conv_layers = nn.ModuleList([GCNConv(num_features, hidden_channels)])
        self.drop_edge_layers = nn.ModuleList([DropEdgeLayer(drop_rate) for _ in range(num_layers - 1)])
        self.conv_layers.extend([GCNConv(hidden_channels, hidden_channels) for _ in range(num_layers - 1)])
        self.final_layer = GCNConv(hidden_channels, num_classes)

    def forward(self, data):
        x, edge_index = data.x, data.edge_index

        # Apply GCN and DropEdge for each layer
        for i in range(self.num_layers):

            if i < self.num_layers - 1:
                edge_index = data.edge_index
                edge_index = self.drop_edge_layers[i](edge_index)

            x = self.conv_layers[i](x, edge_index)
            x = F.relu(x)
            x = F.dropout(x, training=self.training)

        x = self.final_layer(x, edge_index)

        return F.log_softmax(x, dim=1)

In [56]:
dataset = dataset_cite
print('Dataset: CiteSeer')

# Define training parameters
layer_size = 32
epochs = 130
learning_rate = 0.015
drop_rate = 0.3

# Define model
model = GCNWithDropEdge(dataset.num_features, layer_size, dataset.num_classes, drop_rate, num_layers=8)
optimizer = optim.Adam(model.parameters(), lr=learning_rate)
criterion = nn.CrossEntropyLoss()

print(f"Model with hidden size({layer_size}):\n")
print(f"...Training Start...")

# Training
model.train()

for epoch in range(epochs):
    optimizer.zero_grad()
    out = model(dataset)
    loss = criterion(out[dataset.train_mask], dataset.y[dataset.train_mask])
    loss.backward()
    optimizer.step()

    # Calculate training accuracy
    _, predicted = out.max(dim=1)
    correct = predicted[dataset.train_mask] == dataset.y[dataset.train_mask]
    train_acc = correct.sum().item() / dataset.train_mask.sum().item()

    # Printing Every 20 epochs
    if ((epoch+1) % 10) == 0 or epoch+1 == epochs:
        print(f"Epoch {epoch+1}: loss={loss.item()}, training accuracy={train_acc}")

# Evaluation
model.eval()
with torch.no_grad():
    logits = model(dataset)
    _, predicted = logits.max(dim=1)
    correct = predicted[dataset.val_mask] == dataset.y[dataset.val_mask]
    acc = correct.sum().item() / dataset.val_mask.sum().item()

print(f"...Training End...")
print(f"\nValidation accuracy= {acc}")

# Test Evaluation for the Best Model
model.eval()
with torch.no_grad():
    logits = model(dataset)
    _, predicted = logits.max(dim=1)
    correct = predicted[dataset.test_mask] == dataset.y[dataset.test_mask]
    acc = correct.sum().item() / dataset.test_mask.sum().item()

print(f"Test accuracy= {acc}")

Dataset: CiteSeer
Model with hidden size(32):

...Training Start...
Epoch 10: loss=1.746225357055664, training accuracy=0.23333333333333334
Epoch 20: loss=1.6936631202697754, training accuracy=0.38333333333333336
Epoch 30: loss=1.1580437421798706, training accuracy=0.5333333333333333
Epoch 40: loss=1.2659180164337158, training accuracy=0.5416666666666666
Epoch 50: loss=0.9617868065834045, training accuracy=0.5833333333333334
Epoch 60: loss=0.8618496656417847, training accuracy=0.6333333333333333
Epoch 70: loss=0.8151241540908813, training accuracy=0.6416666666666667
Epoch 80: loss=0.7987928986549377, training accuracy=0.6833333333333333
Epoch 90: loss=0.7234742045402527, training accuracy=0.725
Epoch 100: loss=0.7190862894058228, training accuracy=0.6833333333333333
Epoch 110: loss=0.7791874408721924, training accuracy=0.75
Epoch 120: loss=0.7186530828475952, training accuracy=0.75
Epoch 130: loss=0.6421231031417847, training accuracy=0.7333333333333333
...Training End...

Validation a

## **Skip Connection**

In [57]:
class GCNWithSkipConnection(nn.Module):
    def __init__(self, num_features, hidden_channels_list, num_classes):
        super(GCNWithSkipConnection, self).__init__()
        self.num_layers = len(hidden_channels_list) + 1

        self.conv1 = GCNConv(num_features, hidden_channels_list[0])
        self.conv_layers = nn.ModuleList([GCNConv(hidden_channels_list[i - 1], hidden_channels_list[i]) for i in range(1, len(hidden_channels_list))])
        self.final_layer = GCNConv(hidden_channels_list[-1], num_classes)

    def forward(self, data):
        x, edge_index = data.x, data.edge_index

        # Apply GCN for each layer
        x = self.conv1(x, edge_index)
        x = F.relu(x)
        # Add skip connection
        x_old = x
        for i, conv_layer in enumerate(self.conv_layers):
            x = conv_layer(x, edge_index)
            x = F.relu(x)
            # Skip connection
            x = x + x_old
            x_old = x
        x = self.final_layer(x, edge_index)

        return F.log_softmax(x, dim=1)

In [61]:
dataset = dataset_cite
print('Dataset: CiteSeer')

# Define training parameters
layer_size = [64,64,64,64,64,64,64]
epochs = 130
learning_rate = 0.015
drop_rate = 0.3

# Define model
model = GCNWithSkipConnection(dataset.num_features, layer_size, dataset.num_classes)
optimizer = optim.Adam(model.parameters(), lr=learning_rate)
criterion = nn.CrossEntropyLoss()

print(f"Model with hidden size({layer_size}):\n")
print(f"...Training Start...")

# Training
model.train()

for epoch in range(epochs):
    optimizer.zero_grad()
    out = model(dataset)
    loss = criterion(out[dataset.train_mask], dataset.y[dataset.train_mask])
    loss.backward()
    optimizer.step()

    # Calculate training accuracy
    _, predicted = out.max(dim=1)
    correct = predicted[dataset.train_mask] == dataset.y[dataset.train_mask]
    train_acc = correct.sum().item() / dataset.train_mask.sum().item()

    # Printing Every 20 epochs
    if ((epoch+1) % 10) == 0 or epoch+1 == epochs:
        print(f"Epoch {epoch+1}: loss={loss.item()}, training accuracy={train_acc}")

# Evaluation
model.eval()
with torch.no_grad():
    logits = model(dataset)
    _, predicted = logits.max(dim=1)
    correct = predicted[dataset.val_mask] == dataset.y[dataset.val_mask]
    acc = correct.sum().item() / dataset.val_mask.sum().item()

print(f"...Training End...")
print(f"\nValidation accuracy= {acc}")

# Test Evaluation for the Best Model
model.eval()
with torch.no_grad():
    logits = model(dataset)
    _, predicted = logits.max(dim=1)
    correct = predicted[dataset.test_mask] == dataset.y[dataset.test_mask]
    acc = correct.sum().item() / dataset.test_mask.sum().item()

print(f"Test accuracy= {acc}")

Dataset: CiteSeer
Model with hidden size([64, 64, 64, 64, 64, 64, 64]):

...Training Start...
Epoch 10: loss=0.10233097523450851, training accuracy=0.9666666666666667
Epoch 20: loss=0.007990460842847824, training accuracy=1.0
Epoch 30: loss=0.0008284560171887279, training accuracy=1.0
Epoch 40: loss=0.00019354723917786032, training accuracy=1.0
Epoch 50: loss=5.326960308593698e-05, training accuracy=1.0
Epoch 60: loss=3.8297210267046466e-05, training accuracy=1.0
Epoch 70: loss=2.7101610612589866e-05, training accuracy=1.0
Epoch 80: loss=1.98473317141179e-05, training accuracy=1.0
Epoch 90: loss=1.7247510186280124e-05, training accuracy=1.0
Epoch 100: loss=1.5208445802272763e-05, training accuracy=1.0
Epoch 110: loss=1.3643709280586336e-05, training accuracy=1.0
Epoch 120: loss=1.241555128217442e-05, training accuracy=1.0
Epoch 130: loss=1.1339455340930726e-05, training accuracy=1.0
...Training End...

Validation accuracy= 0.59
Test accuracy= 0.594


## **Skip Connection with DropEdge**

In [62]:
class GCNWithResidualAndDropEdge(nn.Module):
    def __init__(self, num_features, hidden_channels_list, num_classes, drop_rate):
        super(GCNWithResidualAndDropEdge, self).__init__()
        self.num_layers = len(hidden_channels_list) + 1
        self.drop_rate = drop_rate

        self.conv1 = GCNConv(num_features, hidden_channels_list[0])
        self.drop_edge1 = DropEdgeLayer(drop_rate)
        self.conv_layers = nn.ModuleList([GCNConv(hidden_channels_list[i - 1], hidden_channels_list[i]) for i in range(1, len(hidden_channels_list))])
        self.drop_edge_layers = nn.ModuleList([DropEdgeLayer(drop_rate) for _ in range(len(hidden_channels_list))])
        self.final_layer = GCNConv(hidden_channels_list[-1], num_classes)

    def forward(self, data):
        x, edge_index = data.x, data.edge_index

        # Apply GCN and residual connection with DropEdge for each layer
        x_old = x
        x = self.conv1(x, edge_index)
        x = F.relu(x)
        # Add DropEdge
        edge_index = self.drop_edge1(edge_index)
        for i, (conv_layer, drop_edge_layer) in enumerate(zip(self.conv_layers, self.drop_edge_layers)):
            edge_index = edge_index
            x_residual = x
            x = conv_layer(x, edge_index)
            x = F.relu(x)
            # Add DropEdge
            edge_index = drop_edge_layer(edge_index)
            # Residual connection
            x = x + x_residual
        x = self.final_layer(x, edge_index)

        return F.log_softmax(x, dim=1)


In [63]:
dataset = dataset_cite
print('Dataset: CiteSeer')

# Define training parameters
layer_size = [64,64,64,64,64,64,64]
epochs = 130
learning_rate = 0.015
drop_rate = 0.3

# Define model
model = GCNWithSkipConnection(dataset.num_features, layer_size, dataset.num_classes)
optimizer = optim.Adam(model.parameters(), lr=learning_rate)
criterion = nn.CrossEntropyLoss()

print(f"Model with hidden size({layer_size}):\n")
print(f"...Training Start...")

# Training
model.train()

for epoch in range(epochs):
    optimizer.zero_grad()
    out = model(dataset)
    loss = criterion(out[dataset.train_mask], dataset.y[dataset.train_mask])
    loss.backward()
    optimizer.step()

    # Calculate training accuracy
    _, predicted = out.max(dim=1)
    correct = predicted[dataset.train_mask] == dataset.y[dataset.train_mask]
    train_acc = correct.sum().item() / dataset.train_mask.sum().item()

    # Printing Every 20 epochs
    if ((epoch+1) % 10) == 0 or epoch+1 == epochs:
        print(f"Epoch {epoch+1}: loss={loss.item()}, training accuracy={train_acc}")

# Evaluation
model.eval()
with torch.no_grad():
    logits = model(dataset)
    _, predicted = logits.max(dim=1)
    correct = predicted[dataset.val_mask] == dataset.y[dataset.val_mask]
    acc = correct.sum().item() / dataset.val_mask.sum().item()

print(f"...Training End...")
print(f"\nValidation accuracy= {acc}")

# Test Evaluation for the Best Model
model.eval()
with torch.no_grad():
    logits = model(dataset)
    _, predicted = logits.max(dim=1)
    correct = predicted[dataset.test_mask] == dataset.y[dataset.test_mask]
    acc = correct.sum().item() / dataset.test_mask.sum().item()

print(f"Test accuracy= {acc}")

Dataset: CiteSeer
Model with hidden size([64, 64, 64, 64, 64, 64, 64]):

...Training Start...
Epoch 10: loss=0.05564705654978752, training accuracy=0.975
Epoch 20: loss=0.006867618765681982, training accuracy=1.0
Epoch 30: loss=0.0009768388699740171, training accuracy=1.0
Epoch 40: loss=0.00030855779186822474, training accuracy=1.0
Epoch 50: loss=9.947941725840792e-05, training accuracy=1.0
Epoch 60: loss=4.334590266807936e-05, training accuracy=1.0
Epoch 70: loss=2.7358388251741417e-05, training accuracy=1.0
Epoch 80: loss=2.383056744292844e-05, training accuracy=1.0
Epoch 90: loss=2.0475650671869516e-05, training accuracy=1.0
Epoch 100: loss=1.7695552742225118e-05, training accuracy=1.0
Epoch 110: loss=1.5693574823671952e-05, training accuracy=1.0
Epoch 120: loss=1.446012538508512e-05, training accuracy=1.0
Epoch 130: loss=1.3347760614124127e-05, training accuracy=1.0
...Training End...

Validation accuracy= 0.61
Test accuracy= 0.607
